In [ ]:
!pip install -q trino

In [ ]:
# This CATALOG_URL works for the "docker compose" testing and development environment
# Change 'server' if you are not running on "docker compose" (f. ex. 'localhost' if Lakekeeper is running locally).
CATALOG_URL = "http://server:8080/catalog"

TRINO_URI = "http://trino:8080"

# In the provided "docker compose" testing and development environment,
# this warehouse already exists.
# See USER_GUIDE.MD and STORAGE.MD for configuration examples if you
# are running this notebook with your own setup.
# There is also an example in 'Multiple Warehouses.ipynb'.
WAREHOUSE = "demo"
CATALOG = "demo_catalog"
NAMESPACE = "demo_namespace"

# Create Trino Catalog

In [ ]:
from trino.dbapi import connect

conn = connect(host=TRINO_URI, user="trino")

In [ ]:
cur = conn.cursor()
cur.execute(
    f"""
    CREATE CATALOG demo USING iceberg
    WITH (
        "iceberg.catalog.type" = 'rest',
        "iceberg.rest-catalog.uri" = '{CATALOG_URL}',
        "iceberg.rest-catalog.warehouse" = '{WAREHOUSE}',
        "iceberg.rest-catalog.security" = 'NONE',
        "iceberg.rest-catalog.vended-credentials-enabled" = 'true',
        "s3.region"= 'dummy',
        "s3.path-style-access" = 'true',
        "s3.endpoint" = 'http://minio:9000',
        "fs.native-s3.enabled" = 'true'
    )
"""
)
cur.execute("CREATE SCHEMA trino_schema")

## Read and Write Tables

In [ ]:
# Connect directly to demo catalog, so that we don't have to use it as a prefix
conn = connect(host=TRINO_URI, user="trino", catalog="demo")
cur = conn.cursor()

In [ ]:
cur.execute("CREATE SCHEMA IF NOT EXISTS trino_namespace")

In [ ]:
cur.execute(
    "CREATE TABLE trino_namespace.my_table (my_ints INT, my_floats DOUBLE, strings VARCHAR) WITH (format='PARQUET')"
)
cur.execute(
    "INSERT INTO trino_namespace.my_table VALUES (1, 1.0, 'a'), (2, 2.0, 'b')"
)